In [10]:
import os
import sys
sys.path.append('D:\实习\研报复现\QuantDev\ymtool')
import numpy as np
import pandas as pd
from tqdm import tqdm
import statsmodels.api as sm

import Alpha as alpha

In [6]:
mv = np.log(alpha.load_data('pv\\stock_mv')).stack()

def neutral(fct):
    res = []
    for idx in tqdm(fct.index):
        temp = pd.concat([fct.loc[idx],mv.loc[idx]],axis=1).dropna()

        X = temp.iloc[:,1:]
        X = sm.add_constant(X)
        y = temp.iloc[:,0]

        # 创建模型
        model = sm.OLS(y, X)

        # 拟合模型
        results = model.fit()

        res.append(results.resid)

    fctNeu = pd.concat(res,axis=1).T
    fctNeu.index = fct.index
    return fctNeu

In [15]:
turnover = alpha.load_data('pv\\stock_turnover').replace(0,np.nan)
STR = turnover.rolling(20,min_periods=1).std().dropna(axis=0,how='all')
SCR = neutral((STR.div(turnover.rolling(40,min_periods=1).std().shift(20),axis=0)-1).dropna(axis=0,how='all').replace([np.inf,-np.inf],np.nan))

100%|█████████████████████████████████████████████████████████████████████████████| 3682/3682 [00:16<00:00, 225.25it/s]


In [16]:
alpha.to_fctDataBase(SCR,'SCR')
fct = alpha.load_fct('SCR')
alpha.Alpha(fct, 'SCR', verbose=True, freq='W', layer=10, rolling_ic=True).main()

开始计算Barra相关性
开始计算RankIC
清除ST
清除涨跌停
获取交易面板完成
开始分层回测...
开始计算分层超额...
结果已经存入D:\实习\研报复现\QuantResearch\选股因子回测结果\report\SCR_W.xlsx
